In [6]:
# Load Python programming language corpus
from datasets import load_dataset

raw_datasets = load_dataset("theothertom/codeparrot-python-only", split="train")

# Check size
print(f"Rows: {len(raw_datasets)}")
print(f"Size: {raw_datasets.data.nbytes / 1e6:.1f} MB")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Rows: 45001
Size: 360.7 MB


In [8]:
import pprint
pprint.pprint(raw_datasets[0])

{'code': 'from django import forms\n'
         'from django.core.exceptions import ValidationError\n'
         'from django.core.validators import validate_slug\n'
         'from django.db import models\n'
         'from django.utils import simplejson as json\n'
         'from django.utils.text import capfirst\n'
         'from django.utils.translation import ugettext_lazy as _\n'
         '\n'
         'from philo.forms.fields import JSONFormField\n'
         'from philo.utils.registry import RegistryIterator\n'
         'from philo.validators import TemplateValidator, json_validator\n'
         '#from philo.models.fields.entities import *\n'
         '\n'
         '\n'
         'class TemplateField(models.TextField):\n'
         '\t"""A :class:`TextField` which is validated with a '
         ':class:`.TemplateValidator`. ``allow``, ``disallow``, and ``secure`` '
         'will be passed into the validator\'s construction."""\n'
         '\tdef __init__(self, allow=None, disallow=None